# Importing packages that will be needed for this notebook

In [1]:
import agama
import numpy
import matplotlib.pyplot as plt
from matplotlib import colors
import subprocess
import os
from mgefit import *
from mgefit.find_galaxy import find_galaxy
from photutils.isophote import Ellipse
from tqdm import trange, tqdm
from astropy.io import fits
from astropy.modeling import models
from petrofit.modeling import fit_model
from petrofit.modeling import print_model_params
from petrofit.modeling import plot_fit

### The 'run' variable determines if a batch of mocks is generated or not

In [2]:
#true if you want to run models, false if not
run = True

## Functions

### makeMock creates the model and outputs it as an N-body snapshot

In [3]:
def makeMock(scaleRad, gamm_a, M_sol = 10**7, M_bh = 10**6.0):

    
    ''' makes spheroid mock with fixed beta = 4, rCut, and alpha = 1
    needs scaleRad and gamma 
    returns density, potential, and snapshot name'''
    rCut = 10*scaleRad
    snapshot_name = "mock_" + "a=" + str(numpy.format_float_positional(scaleRad, precision = 3)) + "_gamma=" + str(numpy.format_float_positional(gamm_a, precision = 3))
    #snapshot_name = "mock_test.txt"
    agama.setUnits(length=1, mass=1, velocity=1)  # 1 Msun, 1 kpc, 1 km/s
    pot_gal = agama.Potential(type='spheroid', mass=M_sol, scaleradius=scaleRad, outercutoffradius=rCut, gamma=gamm_a, beta=4)
    pot_bh  = agama.Potential(type='plummer',  mass=M_bh, scaleradius=1e-4)

    # making the galaxy model
    pot_tot = agama.Potential(pot_gal, pot_bh)
    df = agama.DistributionFunction(type='quasispherical', density=pot_gal, potential=pot_tot, beta0=0.5)  # beta0 is the anisotropy coefficient
    gm = agama.GalaxyModel(pot_tot, df)
    agama.writeSnapshot("snapshots/"+snapshot_name, gm.sample(1000000), 'text')

    galaxydens  = agama.Density(pot_gal)

    return galaxydens, pot_gal, snapshot_name

### getRhalf computes the 2D and 3D half-light radius $r_{1/2}$ of the mock(s) 

In [4]:
def getRhalf(density):

    ''' computes 2 and 3 dimensional half light radius of a mock given its density model from makeMock'''
    
    xyz, mass = density.sample(100000)
    r2d = numpy.sum(xyz[:,0:2]**2, axis=1)**0.5
    order2 = numpy.argsort(r2d)
    cumulmass2 = numpy.cumsum(mass[order2])
    rhalf2d = r2d[order2[numpy.searchsorted(cumulmass2, 0.5*cumulmass2[-1])]]
    r3d = numpy.sum(xyz**2, axis=1)**0.5
    order3 = numpy.argsort(r3d)
    cumulmass3 = numpy.cumsum(mass[order3])
    rhalf3d = r3d[order3[numpy.searchsorted(cumulmass3, 0.5*cumulmass3[-1])]]
    print("r_1/2 2d: ", rhalf2d, "   r_1/2 3d: ", rhalf3d)
    return numpy.format_float_positional(rhalf2d, precision = 4), numpy.format_float_positional(rhalf3d, precision = 4)

### findReff computes the 2D and 3D effective radius $r_e$ of the mock(s)

In [5]:
def findReff(density):
    
    '''finds 2d and 3d effective radius of mock snapshot'''
    
    xyz, mass = density.sample(100000)
    r2d = numpy.sum(xyz[:,0:2]**2, axis=1)**(1/numpy.e)
    order2 = numpy.argsort(r2d)
    cumulmass2 = numpy.cumsum(mass[order2])
    reff2d = r2d[order2[numpy.searchsorted(cumulmass2, 0.5*cumulmass2[-1])]]
    r3d = numpy.sum(xyz**2, axis=1)**(1/numpy.e)
    order3 = numpy.argsort(r3d)
    cumulmass3 = numpy.cumsum(mass[order3])
    reff3d = r3d[order3[numpy.searchsorted(cumulmass3, 0.5*cumulmass3[-1])]]
    print("r_e 2d: ", reff2d, "   r_e 3d: ", reff3d)
    return numpy.format_float_positional(reff2d, precision = 4), numpy.format_float_positional(reff3d, precision = 4)
    #return reff2d, reff3d


### makeImg constructs a FITS image of the mock(s)

In [6]:
def makeImg(pot_gal, snapshot_name):

    '''makes a .fits image of mock snapshot made by makeMock.'''
    # galaxy density
    galaxydens  = agama.Density(pot_gal)


    print('Reading input snapshot')
    snapshot  = agama.readSnapshot("snapshots/"+snapshot_name)
    posvel    = numpy.array(snapshot[0])  # 2d Nx6 array of positions and velocities
    mass      = numpy.array(snapshot[1])  # 1d array of N particle masses

    # convert the N-body model (which was set up in N-body units with G=1) to observational units defined at the beginning of this script
    distance  = 15420                              # [REQ] assumed distance [kpc]
    arcsec2kpc= distance * numpy.pi / 648000        # conversion factor (number of kiloparsecs in one arcsecond)

    rscale = 1/arcsec2kpc   # [REQ] 1 length unit of the N-body snapshot corresponds to this many length units of this script (arcseconds)
    mscale = 1.0            # [REQ] 1 mass unit of the snapshot corresponds to this many mass units of this script (solar masses)
    vscale = 1.0            # (agama.G * mscale / rscale)**0.5  # same for the N-body velocity unit => km/s
    print('1 arcsec=%.3g kpc; vscale=%.3g; units=%s' % (arcsec2kpc, vscale, agama.getUnits()))
    posvel[:, 0:3] *= rscale
    posvel[:, 3:6] *= vscale
    mass *= mscale
    
    nrot = 9  # [OPT] number of rotation angles
    posvel_stack = []
    print('Creating %d rotated copies of input snapshot' % nrot)
    for ang in numpy.linspace(0, numpy.pi, nrot+1)[:-1]:
        sina, cosa = numpy.sin(ang), numpy.cos(ang)
        posvel_stack.append( numpy.column_stack((
            posvel[:,0] * cosa + posvel[:,1] * sina,
            posvel[:,1] * cosa - posvel[:,0] * sina,
            posvel[:,2],
            posvel[:,3] * cosa + posvel[:,4] * sina,
            posvel[:,4] * cosa - posvel[:,3] * sina,
            posvel[:,5] )) )
        
    posvel = numpy.vstack(posvel_stack)
    mass   = numpy.tile(mass, nrot) / nrot

    print("Total Mass ",sum(mass))

    # project to the sky plan
    incl = 45 * numpy.pi/180

    X, Y  = numpy.dot(agama.makeRotationMatrix(alpha=0, beta=incl, gamma=0)[0:2], posvel[:,:3].T)[0:2]  # image-plane coords
    #xmax  = numpy.percentile(numpy.abs(X), 99)
    #ymax  = numpy.percentile(numpy.abs(Y), 99)
    xmax  = 15    # it is the image field of view
    ymax  = 15
    pixel = max(
        numpy.percentile( (X*X+Y*Y)**0.5, 100./len(posvel[:,0]) ),  # choose the pixel size to contain at least ~100 particles in the center,
        max(xmax,ymax) / 100 )    # but don't make it too small, or else we run out of memory in constructing the histogram
    gridx = numpy.linspace(-xmax, xmax, 580)
    gridy = numpy.linspace(-ymax, ymax, 580)
    hist  = numpy.histogram2d(X, Y, bins=(gridx, gridy), weights=mass)[0].T
    scale=2*xmax/len(gridx)
    print('pixel size=', 2*xmax/len(gridx) , 'arcsec')
    print('1 arcsec=', arcsec2kpc, 'kpc')
    print('1 kpc=', 1./arcsec2kpc, 'arcsec') 

    fig, ax1 = plt.subplots(figsize=(6, 6))
    ax1 = plt.axes()

    p=ax1.imshow(numpy.log10(hist), interpolation='nearest', aspect='auto', origin='lower',
    extent=[gridx[0], gridx[-1], gridy[0], gridy[-1]],cmap='Greys_r')
    #ax1.contour(xp, yp, np.log10(H1), colors='k', origin='image')

    #ax1.text(-10, 12, '$ I_{2} $', fontsize=20, color='black')
    ax1.set_xlabel("X [arcsec]")
    ax1.set_ylabel("Y [arcsec]")
    ax1.set_aspect(1)

    # save the image
    print(sum(sum(hist)))
    hdu = fits.PrimaryHDU()
    hdu.data = hist

    hdu.writeto("mock_fits/"+snapshot_name+".fits", overwrite=True)

    print(hist.shape, gridx.shape, gridy.shape)

    return gridx.shape, gridy.shape, scale, snapshot_name+".fits"

### saveParams saves a variety of parameters to a .txt file

In [7]:
def saveParams(outputName, mockArr, gammArr, scaleArr, cutoffArr, reArr, r2Arr, sersArr, fitArr, snapArr, imgArr):

    '''saves params for each mock'''
    '''things to save: 
    gamma, scale rad, cutoffrad, galfit_file_name, '''
    
    '''
    format:
    mock name, gamma, scale radius, cutoff radius, r_eff (2d, 3d), r_1/2 (2d, 3d), 
    galfit file, mock snapshot name, fits name
    '''
    
    head = 'mock  gamma  scale_radius  cutoff_rad  r_eff (2d, 3d)   r_1/2 (2d, 3d)  sers  r_e(fit)  snapshot_name  mock_img_name'

    numpy.savetxt(outputName, numpy.transpose([mockArr, gammArr, scaleArr, cutoffArr, reArr, r2Arr, sersArr, fitArr, snapArr, imgArr]), fmt = "%s %7.3f %7.3f %7.3f %s %s %s %s %s %s", header = head)
    
    return

### findSers determines the Sersic index of 

In [8]:
def findSers(img, plot = False):
    
    '''rHalf = r_eff in pix
    img - data to fit
    plot - will plot data/model/residual'''
    
    hdu = fits.open(img)
    data = hdu[0].data
    numpy.nan_to_num(data, copy=False, nan=0.1, posinf=0.1, neginf=0.1)
    sersic = models.Sersic2D(1, 1, 1, 280, 280, bounds = {"ellip":(0., 1), "theta":(0.,2.)})
    fitted_model, fit_info = fit_model(data, sersic)
    print_model_params(fitted_model)
    params = fitted_model.parameters
    amp = params[0]
    rEff = params[1]
    n = params[2]
    print("Sersic Index of Model: ", n)
    #print("True r_eff [pix]: ", rHalf)
    print("Fitted r_eff [pix]: ", rEff)
    print("############################")
    
    if plot:
        vmax = numpy.std(data)
        vmin = -vmax
        plot_fit(fitted_model, data, vmax=vmax, vmin=vmin, figsize=[3*6, 6])
        plt.show()
    
    return n, rEff 

In [9]:
def __run__(aGrid, gammaGrid, mockName, outputName, plot = True):

    '''does everything
    a grid - grid of scale rads
    gamma grid - grid of inner power law slopes
    baseName - base mock file name
    outputname - name of output file with all params
    '''
    
    mockNames = []
    gammas =[]
    scaleRads = []
    cutoffRads = []
    r_es = []
    r_2s = []
    sersInd = []
    r_e_sersfit = []
    snapshotNames = []
    imageNames = []
    
    #make sure folders are made
    folder_name_s = "snapshots"
    if not os.path.exists(folder_name_s):
        os.makedirs(folder_name_s)
        print(f"Folder '{folder_name_s}' folder created successfully!")
    else:
        print(f"Folder '{folder_name_s}' folder already exists! Continue working!")
        
        
    folder_name_f = "mock_fits"
    if not os.path.exists(folder_name_f):
        os.makedirs(folder_name_f)
        print(f"Folder '{folder_name_f}' folder created successfully!")
    else:
        print(f"Folder '{folder_name_f}' folder already exists! Continue working!")
    

    mockN = 0
    errCount = 0
    #for a, g in tqdm(zip(aGrid, gammaGrid), total = len(aGrid)):
    
    #make mocks and save everything
    for a in tqdm(aGrid):
        for g in gammaGrid:
            
            #make mock
            print("Making mock....")
            print("################")
            cutoffRad = 10*a
            densityGal, potentialGal, snapshotName = makeMock(a, g)
        
            #find reff and r_1/2
            print("Finding r_1/2 and r_eff.....")
            print("################")
            reff2d, reff3d = findReff(densityGal)

            rhalf2d, rhalf3d = getRhalf(densityGal)

            print("2d r_eff: ", reff2d, " 3d r_eff ", reff3d)
            print("2d r_1/2: ", rhalf2d, " 3d r_1/2 ", rhalf3d)
            print("gamma = ", g, "a = ", a)

            #make image with mge
            print("Making image.....")
            print("################")
            xShape, yShape, scale, fitName = makeImg(potentialGal, snapshotName)
            
            
            #find sers index and best fit rEff
            print("Fitting 2D sersic.....")
            print("################")
            
            try:
                n, fitR = findSers("mock_fits/"+fitName)
            except:
                print("Error with sersic fit...")
                n = "err"
                fitR = "err"
                errCount += 1
            
            #stuff to write
            mockNames.append(mockName+"_"+str(mockN))
            gammas.append(g)
            scaleRads.append(a)
            cutoffRads.append(cutoffRad)
            r_es.append((reff2d, reff3d))
            r_2s.append((rhalf2d, rhalf3d))
            sersInd.append(n)
            r_e_sersfit.append(fitR)
            snapshotNames.append(snapshotName)
            imageNames.append(fitName)
        
            mockN += 1
            
    #write everything
    print("Num of fit errors:", + errCount)
    print("Writing output....")
    print("################")
    saveParams(outputName, mockNames, gammas, scaleRads, cutoffRads, r_es, r_2s,
               sersInd, r_e_sersfit, snapshotNames, imageNames)
    #print("Num of fit errors:", + errCount)
    #make plots AL- make later if needed
    #if plot:
    #    continue
    
    return mockNames, gammas, scaleRads, cutoffRads, r_es, r_2s, sersInd, r_e_sersfit, snapshotNames, imageNames

## Run everything

In [10]:
trueScale = 6.4614 #pix
trueScale *= 0.05 #pix x arc/pix = arc
trueScale *= 15420 * numpy.pi / 648000 #arc x kpc/arc = kpc
print("r_1/2 for UCD736:", trueScale, "[kpc]")

r_1/2 for UCD736: 0.024152154168424114 [kpc]


In [12]:
gammaVals = numpy.arange(0.5, 1.75, 0.25)
scaleVals = numpy.linspace(0.003, 0.03, 25)

In [13]:
print(gammaVals)

[0.5  0.75 1.   1.25 1.5 ]


In [ ]:
if run:
    mocks, gamms, scales, cutoffs, res, r2s, sers, refit, snaps, imgs = __run__(scaleVals, gammaVals, "mock_NSC", "mock_params_NSC.txt")